# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
# imports
import os
from dotenv import load_dotenv
from openai import OpenAI
import ollama
import ipywidgets as widgets
from IPython.display import display, Markdown

In [ ]:
# constants

MODEL_GEMINI = "gemini-2.5-flash"
MODEL_LLAMA = "llama3.1:8b"

CHOICE_GEMINI = "gemini"
CHOICE_OLLAMA = "ollama"

SYSTEM_PROMPT = (
    "You are a technical adviser. The student is learning LLM engineering "
    "and you will be asked to explain lines of code with an example, "
    "mostly in Python."
    "You can answer other questions as well."
)

GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

In [ ]:
# set up environment
load_dotenv(override=True)
google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    print("Warning: GOOGLE_API_KEY not found. Gemini calls will fail.")
    print("Please create a .env file with GOOGLE_API_KEY=your_key")

gemini_client = OpenAI(
    base_url=GEMINI_BASE_URL,
    api_key=google_api_key,
)


In [ ]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [ ]:
def make_messages(user_question: str):
    return [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_question},
    ]


def stream_gemini(messages):
    """Stream response chunks from Gemini."""
    stream = gemini_client.chat.completions.create(
        model=MODEL_GEMINI,
        messages=messages,
        stream=True,
    )

    full = []
    for chunk in stream:
        piece = chunk.choices[0].delta.content or ""
        full.append(piece)
    return "".join(full)


def stream_ollama(messages):
    """Stream response chunks from local Ollama."""
    stream = ollama.chat(
        model=MODEL_LLAMA,
        messages=messages,
        stream=True,
    )

    full = []
    for chunk in stream:
        piece = chunk["message"]["content"]
        full.append(piece)
    return "".join(full)


def get_explanation(question: str, model_choice: str):
    """Gets a technical explanation from the chosen model and streams the response."""
    messages = make_messages(question)
    try:
        if model_choice == CHOICE_GEMINI:
            return stream_gemini(messages)
        elif model_choice == CHOICE_OLLAMA:
            return stream_ollama(messages)
        else:
            print("Unknown model choice.")
            return ""
    except Exception as e:
        print(f"\nAn error occurred: {e}")
        return ""

print("💡 Your personal technical tutor is ready.\n")

# Dropdown for model selection
model_dropdown = widgets.Dropdown(
    options=[
        ("Gemini (gemini-2.5-flash)", CHOICE_GEMINI),
        ("Ollama (llama3.1:8b)", CHOICE_OLLAMA),
    ],
    value=CHOICE_GEMINI,
    description="Model:",
    style={"description_width": "initial"},
)

# Text input for question
question_box = widgets.Textarea(
    placeholder="Type your technical question here...",
    description="Question:",
    layout=widgets.Layout(width="100%", height="100px"),
    style={"description_width": "initial"},
)

submit_button = widgets.Button(description="Ask", button_style="success", icon="paper-plane")

output_area = widgets.Output()
loader_label = widgets.Label(value="")

def on_submit(_):
    output_area.clear_output()
    question = question_box.value.strip()
    if not question:
        with output_area:
            print("Please enter a question.")
        return

    loader_label.value = "⏳ Thinking..."
    submit_button.disabled = True

    answer = get_explanation(question, model_dropdown.value)

    loader_label.value = ""
    submit_button.disabled = False

    with output_area:
        print(f"🤖 Model: {model_dropdown.label}")
        print(f"📜 Question: {question}\n")
        display(Markdown(answer))
        print("\n--- End of response ---")

submit_button.on_click(on_submit)

# Display everything
display(model_dropdown, question_box, submit_button, loader_label, output_area)